In [1]:
colab = True # если работаете на своём компьютере, поставьте False
if colab:
    from google.colab import drive
    drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 7.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606632 sha256=28ab590cfc93be1b3ac7794b2287f35921d5f4f16c79ed2ef53b6191b4b9827b
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import itertools
import scipy.sparse as sps

import implicit

In [4]:
!ls /content/drive/'/MyDrive/Environments/recommendations_product_gpn_cup_2021/data'

df_nomencl.parquet			   X_test_sps.npz
df_scores_by_group.parquet		   X_train.parquet
matrix_sku_popularity_above_2000000.csv    X_train_sps_0_1.npz
matrix_sku_popularity_after_2000000.csv    X_train_sps_1_2.npz
matrix_sku_popularity_recommended.parquet  X_train_sps_2_3.npz
nomenclature.parquet			   X_train_sps_3.npz
not_recommended_sku_ids.pkl		   X_train_sps.npz
transactions-for_submission.parquet	   X_valid_sps.npz
X_test_for_pred.parquet			   y_test_for_pred_list.pkl
X_test.parquet				   y_test_for_pred.parquet


### Формирование матрицы рекомендаций

In [ ]:
X_train = pd.read_parquet("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/X_train.parquet")

In [ ]:
X_train.head()

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date,month
7620118,1158,0.002270,0.000000,1764664,0.159197,192548,77,2171-03-01,3
7585450,3324,0.006106,0.000452,2262585,0.000000,345359,96,2171-03-01,3
7585449,3324,0.006106,0.000452,2262672,0.000000,66311,96,2171-03-01,3
7585448,3324,0.006106,0.000452,1764707,0.000000,352887,77,2171-03-01,3
7585447,417,0.008107,0.000452,3289368,0.000000,179887,102,2171-03-01,3


In [ ]:
def create_matrix_sku_popularity(data):
    '''
    Формирует матрицу товары х товары, где в столбцах будут товары из чеков, 
    а в стороках все товары в продаже, в значениях количество упоминаний товара, 
    другими словами сколько и какие товары покупают с товаром из столбца
    '''
    #сформирую шаблон матрицы
    matrix = pd.DataFrame(columns=data.sku_id.sort_values().unique(), 
                      index=data.sku_id.sort_values().unique(),
                      data=0)
    
    #иду по всем чекам из выборки
    for cheque_id in tqdm(data.cheque_id.unique()):
        #формирую список уникальных товаров в текущем чеке
        sku_id_in_cheque = list(data[data['cheque_id'] == cheque_id].sku_id.values)
        #перебираю каждый товар в чеке
        for sku_id in sku_id_in_cheque:
            #формирую список колонок-sku_id которые есть в текущем чеке, за исключением текущего товара
            columns = set(sku_id_in_cheque) - {sku_id}
            #в индексе у меня все товары, ищу такой товар и для совместных товаров в чеке увеличиваю счетчик
            matrix.loc[sku_id][columns] += 1
    
    return matrix

In [ ]:
X_train.shape

(3828486, 10)

In [ ]:
matrix_sku_popularity = create_matrix_sku_popularity(X_train[2000000:]) #разделял на 2 части, одна формировалась тут другая на моем ПК, потом совмещал
#matrix_sku_popularity = create_matrix_sku_popularity(X_train)

100%|██████████| 689721/689721 [1:11:35<00:00, 160.59it/s]


In [ ]:
matrix_sku_popularity

,0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,19,20,21,23,24,25,26,28,29,30,31,32,33,35,36,38,39,40,41,42,43,44,45,...,4936,4937,4938,4939,4940,4941,4942,4944,4947,4951,4952,4953,4954,4956,4957,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991,4992,4993,5012,5013,5014,5015,5017,5023,5036,5037
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5023,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5036,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,1,0,0,0,1,0,0,...,0,0,3,1,0,6,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
matrix_sku_popularity.to_csv("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/matrix_sku_popularity_after_2000000.csv")

###  Подготовка рекомендаций в расширенной таблице

Посмотрю в каких категориях больше всего ошибок предсказания, как качество предсказания зависит от товаров в чеке, если товар один, два, три и т.д.  
Разобью чеки на те в которых есть топливо и в которых нет топлива, далее каждую группу разобью на подгруппы - в зависимости от количество товаров в чеке.

In [5]:
top_20_sku_recommended = [3329,
 3324,
 3334,
 3330,
 2673,
 1551,
 1535,
 98,
 434,
 3336,
 433,
 580,
 808,
 403,
 1533,
 817,
 99,
 229,
 3325,
 431]

In [6]:
#Соберу по примеру выше все в функцию
#На самом деле в функцию уже добавлены новые фичи, но про них я буду описывать ниже

def get_recommended_for_cheque (cheque_id, matrix_sku_popularity_recommended, 
                               sku_ids, df_nomencl, default=top_20_sku_recommended, 
                               top=20, top_sku=100, filter_already_buy_sku=True,
                               group_rotation=False):
    '''
    Собирает таблицу рекомендаций по матрице matrix_sku_popularity для чека cheque_id
    cheque_id - ид чека (одного)
    matrix_sku_popularity_recommended - матрица товары х товары
    sku_ids - товары из чека cheque_id
    df_nomencl - данные с номенклатурой, нужны чтобы выдернуть ид категорий по sku_id
    default-дефолтные топ 20 товаров, которые нужно рекомендовать в случае если нет другой рекомендации
    top - сколько рекомендаций давать
    top_sku - сколько брать рекомендаций по каждому товару до формирования исходной таблицы.
             за счет того, то для каких то товаров может быть мало рекомендаций и одинаковые рекомендации схлопываются
             может получиться так, что итоговый топ будет меньше 20, поэтому тут рекомендуется брать больше
             с другой стороны большой top_sku негативно влияет на group_rotation, опытным путем установил 100
    filter_already_buy_sku - не рекомендовать товары из категорий, которые уже есть в чеке (например вода есть, любую воду не рекомендуем)
    group_rotation - включает ротацию топ товаров по категориям, Например в предварительной рекомендации сначала идет несколько вариантов 
                     кофе и только потом предлагаются другие товары, нужно составить рейтинг товаров по категориям и предлагать 
                     по очереди по одному самому популярному товару каждой категории (кофе-вода-снеки и т.д.).
    '''

    #посмотрю по подготовленной матрице, что с этим товаром часто покупают. топ 20
    dfs_sku_recommended = [] #датафреймы рекомендаций по каждому товару в чеке
    #формирую таблицы рекомендаций по каждому товару в чеке
    for sku_id in sku_ids:
        try : #если такого товара нет в моей матрице, то пропускаем его
            _ = matrix_sku_popularity_recommended[str(sku_id)] #проверка что товар(столбец) существует
            
            dfs_sku_recommended.append(matrix_sku_popularity_recommended[[str(sku_id), 'sku_group_id']].
                            sort_values(by=str(sku_id), ascending=False).reset_index().head(top_sku))
        except Exception as ex:
            pass
    #Нужно все эти рекомендации собрать в одну таблицу
    df_sku_recommended = [] #итоговая таблица рекомендации
    for i, df_sku in enumerate(dfs_sku_recommended):
        if i == 0: #если первая таблица рекомендации
            df_sku_recommended = df_sku
        else: #если у нас в чеке больше 1 позиции, перебираем остальные таблицы рекомендации для каждого товара, объединяю по sku_id
            df_sku_recommended = pd.merge(df_sku_recommended, df_sku, on='sku_id', how="outer",
                                          suffixes= ('_x'+str(i), '_y'+str(i))).fillna(0)
    #если есть рекомендации, то
    if len(df_sku_recommended) > 0:
        #теперь нужно сделать новый столбец с суммой рекомендаци по одинаковым товарам из отдельныйх рекомендаций по каждой позиции в чеке
        #нужно чтобы объединить одинаковые товары из таблиц рекомендаций
        sum_col = [col for col in df_sku_recommended.columns if "sku" not in col] #строки которые буду суммировать
        df_sku_recommended['sum_rec'] = df_sku_recommended[sum_col].sum(axis=1)
        df_sku_recommended = df_sku_recommended.sort_values(by='sum_rec', ascending=False)
        #если нужно исключить из рекомендаций группы товаров которые уже есть в чеке
        if filter_already_buy_sku:
            #список исключений групп товаров, из текущего чека, которые клиент уже купил, возможно нет смысла их рекомендовать
            #анализ выше показал, что редко берут 2 позиции воды или других товаров из одной категории
            #даже если клиент захочет 2 позиции, пусть берет, главное попасть в то что он хочет "воду"
            sku_group_not_recommended = list(df_nomencl[df_nomencl.sku_id.isin(sku_ids)].sku_group_id.values) #группы для исключения
            #беру первую колонку с содержанием sku_group_id, формирую список из ид групп для исключения
            col_group_id_name = [col for col in df_sku_recommended.columns if "sku_group_id" in col][0]
            #убираю из рекомендаций категории товаров, которые соответствуют категории из чека
            df_sku_recommended = df_sku_recommended[~df_sku_recommended[col_group_id_name].isin(sku_group_not_recommended)]
            df_sku_recommended = df_sku_recommended.sort_values(by='sum_rec', ascending=False)
            #итоговая таблица рекомендаций к текущему чеку, сортировка по популярности
        #если включена ротация по категориям
        if group_rotation:
            new_top_20_sku_recommended = [] #новый топ 20 товаров который будет сформирован
            top_assembled = 0 #счетчик для топа 20 товаров
            for t in range(0, top):
                if top_assembled == top:
                    break
                #беру первую колонку(все колонки одного содержания) с содержанием sku_group_id
                col_group_id_name = [col for col in df_sku_recommended.columns if "sku_group_id" in col][0]
                #перебираю все группы товаров из предварительной рекомендации
                for group in list(df_sku_recommended[col_group_id_name].unique()):
                    sku_to_append = df_sku_recommended[df_sku_recommended[col_group_id_name] == group]
                    if len(sku_to_append) > t:
                        sku_to_append = sku_to_append.iloc[t].sku_id
                        new_top_20_sku_recommended.append(sku_to_append)
                        top_assembled += 1
                    if top_assembled == 20:
                        break
            #формирую новый датафрейм с топ 20 товаров с ротацией по категориям
            df_sku_recommended = pd.DataFrame({'sku_id': new_top_20_sku_recommended})
    else:#если нет рекомендации
        df_sku_recommended = pd.DataFrame({'sku_id': default}) #холодный страт - рекомендуем просто топ товаров
    
    #итоговая таблица с рекомендациями
    df_submission = pd.DataFrame.from_dict({'cheque_id':[cheque_id]*top, 
                                            'sku_id': df_sku_recommended.sku_id.values[:top],
                                            'position': range(1,top+1)
                                           })
    return df_submission

In [7]:
X_test_for_pred = pd.read_parquet("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/X_test_for_pred.parquet")
y_test_for_pred = pd.read_parquet("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/y_test_for_pred.parquet")
df_nomencl = pd.read_parquet("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/df_nomencl.parquet")
matrix_sku_popularity_recommended = pd.read_parquet("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/matrix_sku_popularity_recommended.parquet")

In [ ]:
#товары топливо
fuel_sku_id = df_nomencl[df_nomencl["sku_group"].str.contains("Топливо")].sku_id.to_list()

In [ ]:
#Сформирую тестовый набор данных
#чеки где есть топливо
fuel_cheque_id = X_test_for_pred[X_test_for_pred.sku_id.isin(fuel_sku_id)].cheque_id.unique()

#данные по транзакциям где есть чеки с топливом
df_sku_fuel = X_test_for_pred[X_test_for_pred.cheque_id.isin(fuel_cheque_id)]

#данные по транзакциям где есть чеки с топливом
df_sku_without_fuel = X_test_for_pred[~X_test_for_pred.cheque_id.isin(fuel_cheque_id)]

In [ ]:
#добавлю номер категории
df_sku_fuel = pd.merge(df_sku_fuel, df_nomencl[['sku_id', 'sku_group_id']])
df_sku_without_fuel = pd.merge(df_sku_without_fuel, df_nomencl[['sku_id', 'sku_group_id']])

In [ ]:
#датафрейм с оценками рекомендаций, разбито по категориям
#fuel - есть в чеке топливо или нет
#curr_rec_group_id - группа товаров из рекомендуемых которая есть в чеке (она точно и может еще какие то)
#groups_id - какие группы товаров присутствует в чеке кроме curr_rec_group_id
#all_sku_count_in_cheque - сколько всего товаров в чеке
#score - оценка рекомендаций на тест
df_scores_by_group = pd.DataFrame(columns=['fuel', 'curr_rec_group_id', 'groups_id', 'all_sku_count_in_cheque', 'score'])
columns = ['sku_id', 'cheque_id', 'sku_group_id', 'shop_id']

for i, df in enumerate([df_sku_fuel[columns].sort_values(by='cheque_id'), 
                        df_sku_without_fuel[columns].sort_values(by='cheque_id')]):
    for group in tqdm(df.sku_group_id.unique()):
        df_group_recs = [] #массиф датафреймов рекомендаций для группы товаров
        y_test = [] #массив ответов для оценки точности рекомендаций
        group_score = 0 #оценка рекомендации для группы товаров
        #все чеки текущей группы
        df_curr_group = df[df.sku_group_id == group]
        #чеки для рекомендаций, отсортированные по количеству товаров в ниж
        cheque_ids_for_rec = list(df_curr_group.sort_values(by='cheque_id').groupby('cheque_id')['sku_id'].count()\
                           .reset_index().sort_values(by='sku_id').cheque_id)
        #формирую рекомендацию для каждого чека
        for cheque_id in cheque_ids_for_rec:
            #группы товаров в чеке
            df_curr_cheque = df[(df.cheque_id == cheque_id)]
            groups_in_cheque = list(df_curr_cheque.sku_group_id.unique())
            # убираю текущую группу
            groups_in_cheque.remove(group)
            #если массив с группами пустой, то оставляю пустое значение,
            if len(groups_in_cheque) < 1:
                groups_in_cheque = ''
            else:
                groups_in_cheque = ','.join(map(str, groups_in_cheque))
            #ид товаров в чеке
            sku_ids = list(df[df['cheque_id'] == cheque_id].sku_id.values)
            #формирую рекомендации для sku_ids из текущего чека
            df_group_recs.append(get_recommended_for_cheque(cheque_id, matrix_sku_popularity_recommended, sku_ids, 
                                             df_nomencl[['sku_id','sku_group_id']], top=20, filter_already_buy_sku=False))
            #ответ на текущий чек, товар какой товар был в чеке дальше, клонирую 20 раз для подачи в функции оценки и сохраняю 
            y_test += list(y_test_for_pred[y_test_for_pred.cheque_id == cheque_id].sku_id.values) *20
        
        #формирую одну таблицу с рекомендациями
        df_rec = pd.concat(df_group_recs)
        
        #Оценка метрики на основе матрицы рекомендаций
        group_score = get_map(y_test, list(df_rec.sku_id.values))
        
        #сохраняю результаты в датафрейм, добавляю новую строчку в конец
        df_scores_by_group.loc[len(df_scores_by_group.index)] = [i, 
                                                                 group, 
                                                                 groups_in_cheque,
                                                                 len(sku_ids),
                                                                 group_score
                                                                ]

  4%|▍         | 1/25 [02:27<58:49, 147.08s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [ ]:
df_scores_by_group.head()

,fuel,curr_rec_group_id,groups_id,all_sku_count_in_cheque,shop_id,score
0,0,15,"23,27,32",8,84,0.011124
1,0,27,,1,107,0.006562
2,0,31,"20,32,28,3",12,19,0.004607
3,0,28,,1,107,0.009152
4,0,25,28,7,73,0.006087


In [ ]:
df_scores_by_group.to_parquet("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/df_scores_by_group.parquet")

#### Подготовка рекомендация для submission

In [8]:
df_sub = pd.read_parquet("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/df_sub.parquet")

In [9]:
#формирую рекомендации для submission

df_rec_sub = [] #массив таблиц с рекомендациями по всем чекам

for cheque_id in tqdm(list(df_sub.cheque_id.unique())):
    sku_ids = list(df_sub[df_sub['cheque_id'] == cheque_id].sku_id.values) #ид товаров в чеке
    df_rec_sub.append(get_recommended_for_cheque(cheque_id, matrix_sku_popularity_recommended, sku_ids, 
                                             df_nomencl[['sku_id','sku_group_id']], top=20, filter_already_buy_sku=False))

  2%|▏         | 5561/315228 [01:01<54:44, 94.29it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 315228/315228 [1:00:14<00:00, 87.22it/s] 


NameError: ignored

In [14]:
#формирую одну таблицу с рекомендациями
df_rec_sub = pd.concat(df_rec_sub)

In [12]:
df_rec_sub.shape

(6304560, 3)

In [13]:

df_rec_sub.to_parquet("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/submission-d_denis.parquet")

##### Test

In [ ]:
X_test = pd.read_parquet("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/X_test.parquet")

In [ ]:
#Подготавливаю матрицу товары(столбцы) х чеки(строки) для обучения
X_test_sps = create_sku_cheque_matrix(X_test, df_nomencl.sku_id.sort_values().unique())

100%|██████████| 125361/125361 [08:32<00:00, 244.72it/s]


In [ ]:
X_test_sps.shape

(125361, 5103)

In [ ]:
sps.save_npz("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/X_test_sps", X_test_sps)

### Подготовка данных для обучения моделей

In [ ]:
def create_sku_cheque_matrix(data, sku_id_list, sparse=True):
    '''
    Формирует матрицу где в столбцах заданные товары, в строках чеки, в значениях количество товаров в чеке
    data - данные с транзакциями на основании которых заполняется матрица
    sku_id_list - шапка матрицы, ид товаров
    sparse - Если true, то формируется sparse матрица, иначе DataFrame
    '''
    #уникальные чеки
    cheque_id_list = data.cheque_id.sort_values().unique()
    #сформирую шаблон матрицы (0 строк, столбцов сколько в data)
    matrix = sps.csr_matrix(np.zeros([0, len(sku_id_list)])).tocsr()
    #по сути это будет одностроковый датафрейм, нужен чтобы свормировать строку для спарс матрицы
    #одностроковыя, чтобы не съедать ОЗУ
    matrix_df = pd.DataFrame(columns=sku_id_list, 
                      index=[0])
    
    row_tmp = dict() #шаблон строки для добавления в матрицу
    for sku_id in sku_id_list: #инициализирую нулями
        row_tmp[sku_id]  = 0 
    
    #иду по всем чекам из выборки
    for i, cheque_id in enumerate(tqdm(cheque_id_list)):
        #формирую список уникальных товаров в текущем чеке
        sku_id_in_cheque = list(data[data['cheque_id'] == cheque_id].sku_id.values)
        row = row_tmp.copy() #строка для текущего чека (словарь, в котором ключ это товары, а значения количество товара в чеке
        #перебираю каждый товар в чеке
        for sku_id in sku_id_in_cheque:
            #заполняю словарь, где в ключах товары, а в значениях сколько раз встречались в чеке
            if row.get(sku_id) is not None:
                row[sku_id] = row[sku_id] + 1 
            #добавляю строку в датафрейм, там где ид товаров совпадает с ключами словаря
            #проставляются значения, во всех остальных будут нули
        matrix_df.loc[0] = list(row.values())
        matrix = sps.vstack([matrix, sps.csr_matrix(matrix_df.loc[0].values.tolist())])
    return matrix

In [ ]:
df_nomencl = pd.read_parquet("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/nomenclature.parquet")

##### Train

In [ ]:
X_train = pd.read_parquet("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/X_train.parquet")
#X_train.to_parquet("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/X_train.parquet")

In [ ]:
X_train.shape

(3828486, 9)

In [ ]:
X_train = X_train[['cheque_id', 'sku_id']]

In [ ]:
#Подготавливаю матрицу товары(столбцы) х чеки(строки) для обучения
X_train_sps = create_sku_cheque_matrix(X_train[3000000:], df_nomencl.sku_id.sort_values().unique())

100%|██████████| 316755/316755 [34:18<00:00, 153.91it/s]


In [ ]:
X_train_sps.shape

(379374, 5103)

In [ ]:
sps.save_npz("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/X_train_sps", X_train_sps)

### Перебор параметров моделей

In [ ]:
open_file = open("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/y_test_for_pred_list.pkl", "rb")
y_test_for_pred_list = pickle.load(open_file)
open_file.close()

In [ ]:
open_file = open("/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/not_recommended_sku_ids.pkl", "rb")
not_recommended_sku_ids = pickle.load(open_file)
open_file.close()

In [ ]:
X_train_sps = sps.load_npz('/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/X_train_sps.npz')
X_valid_sps = sps.load_npz('/content/drive/MyDrive/Environments/recommendations_product_gpn_cup_2021/data/X_valid_sps.npz')

#### Метрики

In [ ]:
def average_precision(actual, recommended, k=20):
    ap_sum = 0
    hits = 0
    for i in range(k):
        product_id = recommended[i] if i < len(recommended) else None
        if product_id is not None and product_id in actual:
            hits +=1
            ap_sum += hits / (i + 1)
    return ap_sum / k

In [ ]:
def get_map(actual, recommended, k=20):
    '''
    Рассчет метрики MAP
    actual-список реальных товаров
    recommended-список рекомендованных товаров
    k-сколько рекомендаций сверять
    '''
    scores = []
    ap = 0
    for i in range(0, int(len(recommended)/20)):
        if i == 0:
            ap = average_precision(actual[:k],
                                   recommended[:k],  
                                             k=k)
        else:
            ap = average_precision(actual[(i-1)*k:(i*k)], 
                              recommended[(i-1)*k:(i*k)], 
                                             k=k)
        scores.append(ap)
    return np.mean(scores)

In [ ]:
#Перебор параметров обучения и проверки моделей
#Не смог тут адаптировать GridSearchCV, поэтому пишу свою функцию
def grid_search_model (model, param_model, param_valid, X_train, X_valid, y_valid=y_test_for_pred_list, transposition=True):
    #параметры построения модели
    keys, values = zip(*param_model.items())
    experiments = [dict(zip(keys, v)) for v in itertools.product(*values)]
    
    #параметры валидации
    keys, values = zip(*param_valid.items())
    validations = [dict(zip(keys, v)) for v in itertools.product(*values)]
    
    best_param = {'param_model':'', 'param_valid':'', 'score':0.0}
    
    if transposition: #если нужно транспонировать данные
        X_train = X_train.T
        
    for experiment in experiments:
        model_ = model(**experiment)
        model_.fit(X_train)
        
        for num, validation in enumerate(validations):
            print('=== model:{} valid[{}]: {} ==='.format(experiment, num, validation))
            recs = [] #все рекомендации
            for i in range(0, X_valid.shape[0]): 
                row = X_valid[i].astype(float)
                rec = model_.recommend(userid=0, 
                              user_items=row, 
                              N=20, 
                              filter_items=not_recommended_sku_ids, 
                              **validation)
                recs += [x[0] for x in rec]
            score = float(get_map(y_valid, recs))
            print('     score[{}]: {}'.format(num, score))
            if score > best_param['score']:
                best_param['param_model'] = experiment
                best_param['param_valid'] = validation
                best_param['score'] = score
    return best_param

##### ASL

In [ ]:
param_model = {
    "factors": [16, 32, 64, 256],
    "regularization": [0.0, 0.1],
    "iterations": [16, 32, 64],
    "use_gpu":[False]
    }

param_valid = {
    "filter_already_liked_items": [True, False],
    "recalculate_user": [True, False]
    }

In [ ]:
grid_search_model(implicit.als.AlternatingLeastSquares, 
                  param_model, param_valid, X_train_sps[:20000], X_valid_sps, transposition=True)

  0%|          | 0/16 [00:00<?, ?it/s]

=== model:{'factors': 16, 'regularization': 0.0, 'iterations': 16, 'use_gpu': False} valid[0]: {'filter_already_liked_items': True, 'recalculate_user': True} ===
     score[0]: 0.1858273652468903
=== model:{'factors': 16, 'regularization': 0.0, 'iterations': 16, 'use_gpu': False} valid[1]: {'filter_already_liked_items': True, 'recalculate_user': False} ===
     score[1]: 0.11398416886543536
=== model:{'factors': 16, 'regularization': 0.0, 'iterations': 16, 'use_gpu': False} valid[2]: {'filter_already_liked_items': False, 'recalculate_user': True} ===
     score[2]: 0.18571428571428572
=== model:{'factors': 16, 'regularization': 0.0, 'iterations': 16, 'use_gpu': False} valid[3]: {'filter_already_liked_items': False, 'recalculate_user': False} ===
     score[3]: 0.1134941575574821


  0%|          | 0/32 [00:00<?, ?it/s]

=== model:{'factors': 16, 'regularization': 0.0, 'iterations': 32, 'use_gpu': False} valid[0]: {'filter_already_liked_items': True, 'recalculate_user': True} ===
     score[0]: 0.1850358085186581
=== model:{'factors': 16, 'regularization': 0.0, 'iterations': 32, 'use_gpu': False} valid[1]: {'filter_already_liked_items': True, 'recalculate_user': False} ===
     score[1]: 0.11978891820580474
=== model:{'factors': 16, 'regularization': 0.0, 'iterations': 32, 'use_gpu': False} valid[2]: {'filter_already_liked_items': False, 'recalculate_user': True} ===
     score[2]: 0.185638899359216
=== model:{'factors': 16, 'regularization': 0.0, 'iterations': 32, 'use_gpu': False} valid[3]: {'filter_already_liked_items': False, 'recalculate_user': False} ===
     score[3]: 0.11971353185073502


  0%|          | 0/64 [00:00<?, ?it/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]

=== model:{'factors': 256, 'regularization': 0.1, 'iterations': 32, 'use_gpu': False} valid[0]: {'filter_already_liked_items': True, 'recalculate_user': True} ===
     score[0]: 0.06931775348661892
=== model:{'factors': 256, 'regularization': 0.1, 'iterations': 32, 'use_gpu': False} valid[1]: {'filter_already_liked_items': True, 'recalculate_user': False} ===
     score[1]: 0.04866189219751225
=== model:{'factors': 256, 'regularization': 0.1, 'iterations': 32, 'use_gpu': False} valid[2]: {'filter_already_liked_items': False, 'recalculate_user': True} ===
     score[2]: 0.07071240105540898
=== model:{'factors': 256, 'regularization': 0.1, 'iterations': 32, 'use_gpu': False} valid[3]: {'filter_already_liked_items': False, 'recalculate_user': False} ===
     score[3]: 0.049114210327930646


  0%|          | 0/64 [00:00<?, ?it/s]

=== model:{'factors': 256, 'regularization': 0.1, 'iterations': 64, 'use_gpu': False} valid[0]: {'filter_already_liked_items': True, 'recalculate_user': True} ===
     score[0]: 0.05672823218997362
=== model:{'factors': 256, 'regularization': 0.1, 'iterations': 64, 'use_gpu': False} valid[1]: {'filter_already_liked_items': True, 'recalculate_user': False} ===
     score[1]: 0.04862419901997738
=== model:{'factors': 256, 'regularization': 0.1, 'iterations': 64, 'use_gpu': False} valid[2]: {'filter_already_liked_items': False, 'recalculate_user': True} ===
     score[2]: 0.05865058424425179
=== model:{'factors': 256, 'regularization': 0.1, 'iterations': 64, 'use_gpu': False} valid[3]: {'filter_already_liked_items': False, 'recalculate_user': False} ===
     score[3]: 0.049189596683000374


{'param_model': {'factors': 16,
  'iterations': 64,
  'regularization': 0.1,
  'use_gpu': False},
 'param_valid': {'filter_already_liked_items': False,
  'recalculate_user': True},
 'score': 0.18635506973237845}